In [ ]:
#!/Users/florian/anaconda3/bin/python
import bs4
import requests
import genanki
import sys
import PyPDF2
import pandas as pd
import pytesseract
from PIL import Image
from googletrans import Translator
import re


In [ ]:
#Function to translate to english
def translate(x):
    try:
        translator = Translator()
        trans = translator.translate(x, dest = 'en')
        return trans.text
    except:
        return None


In [ ]:
# Open the image file
image = Image.open('test vocabulary 2_1.jpg')
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
# Perform OCR using PyTesseract
text = pytesseract.image_to_string(image, lang='deu')
pattern = r'\d+|/|\.b|\.a|\.c|\.|Start|der|die|das'
text = re.sub(pattern, '', text)
# Text uprava
text_list = text.splitlines()
#zbavit se prazdnych stringu
text_ohneempty = [i for i in text_list if len(i)>=2]
#zbavit se stringu s cisly

text_final = pd.DataFrame(text_ohneempty)
text_final.columns = ['German']

# Print the extracted text
# print(text_final)

text_final['English'] = text_final['German'].apply(lambda x: translate(x))
#print(text_final)
#display(text_final)
items = []

for index, row in text_final.iterrows():
    german = row['German']
    english = row['English']
    items.append((german, english))

my_model = genanki.Model(
  1607392319,
  'Simple Model',
  fields=[
    {'name': 'german'},
    {'name': 'english'},
  ],
  templates=[
      {
          'name': 'Forward Card',
          'qfmt': '{{german}}',
          'afmt': '{{FrontSide}}<hr id="answer">{{english}}',
      },
      {
          'name': 'Reverse Card',
          'qfmt': '{{english}}',
          'afmt': '{{FrontSide}}<hr>{{german}}'
      }
  ])

my_deck = genanki.Deck(2059400110, "MH Booklet 1")

for i in items:
    my_deck.add_note(genanki.Note(
        model=my_model,
        fields=i))

genanki.Package(my_deck).write_to_file('output.apkg')
    